In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import data from csv file
cancer_df = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
# view first 5 rows
cancer_df.head(5)

In [ ]:
# check for misisng data and type of each feature 
cancer_df.info()

In [ ]:
# drop id column and Unnamed: 32
cancer_df.drop(["id", "Unnamed: 32"], axis=1, inplace=True)

In [ ]:
# view first 5 rows
cancer_df.head(5)

In [ ]:
# Label encoding
cancer_df['diagnosis'] = LabelEncoder().fit_transform(cancer_df['diagnosis'])

In [ ]:
# view first 5 rows
cancer_df.head(100)

In [ ]:
# seperate features and target
Data = cancer_df.drop(["diagnosis"], axis=1).values
target = cancer_df["diagnosis"]

In [ ]:
# scale data
Data = MinMaxScaler().fit_transform(Data)

In [ ]:
# split data for training and testing
D_train, D_test, t_train, t_test = train_test_split(Data, target, test_size=0.3, random_state=30)

In [ ]:
D_train.shape, t_train.shape

In [ ]:
D_test.shape, t_test.shape

In [ ]:
cv_method = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=30)

In [ ]:
param_svm = {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'kernel':['rbf','linear']}

In [ ]:
grid_svm = GridSearchCV(estimator=SVC(),
                       param_grid=param_svm,
                       scoring='accuracy',
                       cv=cv_method,
                       return_train_score=True)

In [ ]:
grid_svm.fit(D_train, t_train)

In [ ]:
grid_svm.best_params_

In [ ]:
grid_svm.best_score_

In [ ]:
param_tree = {'criterion': ['gini', 'entropy'],
             'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

grid_tree = GridSearchCV(estimator=DecisionTreeClassifier(),
                       param_grid=param_tree,
                       scoring='accuracy',
                       cv=cv_method,
                       return_train_score=True)

In [ ]:
grid_tree.fit(D_train, t_train)

In [ ]:
grid_tree.best_params_

In [ ]:
grid_tree.best_score_

In [ ]:
# Selected SVM for prediction task with C = 10, Kernal = Linear
svm_classifier = SVC(C=10, kernel="linear", random_state=30)
svm_classifier.fit(D_train, t_train)

In [ ]:
svm_predicted = svm_classifier.predict(D_test)

In [ ]:
svm_classifier.score(D_test, t_test)

In [ ]:
plot_confusion_matrix(svm_classifier, D_test, t_test)